In [5]:
import os
import json
import numpy as np
import pandas as pd
from pprint import pprint
from itertools import combinations, product

#sklearn
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, MinMaxScaler
from sklearn.metrics import precision_recall_curve, auc, roc_curve, recall_score, precision_score, f1_score, confusion_matrix

#graph
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
import shap


In [2]:

# %config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
from xgboost import XGBClassifier

import matplotlib as matplot
import matplotlib.pyplot as plt
# %matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings, os 
# warnings.filterwarnings("ignore")

# from keras import Sequential
# from keras.models import Model, load_model
# from keras.layers import *
# from keras.callbacks import ModelCheckpoint
# from keras import regularizers
from matplotlib import pyplot as plt

from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA, TruncatedSVD, PCA
from sklearn.svm import LinearSVC

import xgboost
# from mlxtend.classifier import EnsembleVoteClassifier 
import shap
import sklearn.neighbors

from deepod.models import PReNet
from collections import Counter
from modAL.models import ActiveLearner, Committee

import copy

from sklearn.datasets import make_classification
from alipy import ToolBox

import joblib


In [6]:
features = ['Src IP', 'Dst IP','Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
    'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
    'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
    'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std',
    'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std',
    'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean',
    'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot',
    'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
    'Fwd PSH Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
    'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
    'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'PSH Flag Cnt',
    'ACK Flag Cnt', 'URG Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
    'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Byts',
    'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
    'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean',
    'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std',
    'Idle Max', 'Idle Min']
def clean_df(df):
    # Remove the space before each feature names
    df = df[features].copy()
    df.columns = df.columns.str.strip()
    print('dataset shape', df.shape)
    

    # This set of feature should have >= 0 values
    num = df._get_numeric_data()
    num[num < 0] = 0

    zero_variance_cols = []
    for col in df.columns:
        if len(df[col].unique()) == 1:
            zero_variance_cols.append(col)
    df.drop(zero_variance_cols, axis = 1, inplace = True)
    print('zero variance columns', zero_variance_cols, 'dropped')
    print('shape after removing zero variance columns:', df.shape)

    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    print(df.isna().any(axis = 1).sum(), 'rows dropped')
    df.dropna(inplace = True)
    print('shape after removing nan:', df.shape)

    # Drop duplicate rows
    df.drop_duplicates(inplace = True)
    print('shape after dropping duplicates:', df.shape)

    column_pairs = [(i, j) for i, j in combinations(df, 2) if df[i].equals(df[j])]
    ide_cols = []
    for column_pair in column_pairs:
        ide_cols.append(column_pair[1])
    df.drop(ide_cols, axis = 1, inplace = True)
    print('columns which have identical values', column_pairs, 'dropped')
    print('shape after removing identical value columns:', df.shape)
    return df

def sample_df(curr_df, anomaly_rate):
    num_benign = len(curr_df.loc[curr_df['Label'] == "BENIGN"])
    num_attack = len(curr_df) - num_benign
    ratio = num_attack / num_benign
    
    if ratio > anomaly_rate:
        sample = anomaly_rate * num_benign / num_attack
        sampled_df = pd.concat([curr_df[curr_df['Label'] == 'BENIGN'], 
                                curr_df[curr_df['Label'] != 'BENIGN'].sample(frac=sample, random_state=42)]) 
    else:
        
        sample = (1/anomaly_rate) * num_attack / num_benign
        sampled_df = pd.concat([curr_df[curr_df['Label'] != 'BENIGN'], 
                                curr_df[curr_df['Label'] == 'BENIGN'].sample(frac=sample, random_state=42)]) 
    
    new_ratio = sampled_df.loc[sampled_df["Label"] == "BENIGN"].shape[0] / sampled_df.loc[sampled_df["Label"] != "BENIGN"].shape[0]
    
    return sampled_df



In [7]:
dataset_csv_path = './Dataset/SimulatedCVE/cicflowmeter_cve/'
folder_names = ['dripper/', 'BENIGN/', 'bonesi/']

def read_csv():
    full_df = pd.DataFrame()
    
    for folder in folder_names:
        csv_file_names = os.listdir("Dataset/SimulatedCVE/cicflowmeter_cve/" + folder)
        complete_paths = []
        for csv_file_name in csv_file_names:
            print(csv_file_name)
            complete_paths.append(os.path.join(dataset_csv_path+folder, csv_file_name))
        df = pd.concat(map(pd.read_csv, complete_paths), 
                                ignore_index = True)
    
        df["Label"] = folder[:-1]
        full_df = pd.concat([full_df, df], axis=0, ignore_index=True)
    cleaned_df = clean_df(full_df)
    return full_df
full_df = read_csv()
full_df["Label"].value_counts()


merged_20240809091803_ISCX.csv
merged_20240809091912_ISCX.csv
merged_20240809091837_ISCX.csv
merged_20240809091950_ISCX.csv
merged_20240809092024_ISCX.csv
merged_20240809090239_ISCX.csv
merged_20240809090513_ISCX.csv
merged_20240809090443_ISCX.csv
merged_20240809090412_ISCX.csv
merged_20240809090106_ISCX.csv
merged_20240809090544_ISCX.csv
merged_20240809090035_ISCX.csv
merged_20240809090615_ISCX.csv
final_merged.csv
merged_20240809090137_ISCX.csv
merged_20240809090005_ISCX.csv
merged_20240809090341_ISCX.csv
merged_20240809090310_ISCX.csv
merged_20240809090208_ISCX.csv
merged_20240813111613_ISCX.csv
dataset shape (232661, 63)
zero variance columns [] dropped
shape after removing zero variance columns: (232661, 63)
301 rows dropped
shape after removing nan: (232360, 63)
shape after dropping duplicates: (166106, 63)
columns which have identical values [] dropped
shape after removing identical value columns: (166106, 63)


Label
dripper    152953
BENIGN      47755
bonesi      31953
Name: count, dtype: int64

In [8]:
import random

random.seed(42)  # Set seed for random operations
np.random.seed(42)  # Set seed for NumPy operations
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [9]:
flow_df = full_df.drop(columns=["Flow ID","Src IP","Src Port","Dst IP","Dst Port","Timestamp"], inplace=False, axis=1).copy()
label = flow_df['Label']

# 2D-PCA visualization

In [ ]:
# # subsample_df = sample_df(normalized_df, 1.0)
# subsample_df = normalized_df.copy()
# subsample_df["Label"].value_counts()

: 

: 

In [ ]:
# # subsample_df = flow_df.groupby('Label').apply(pd.DataFrame.sample, frac = 0.1).reset_index(drop = True)
# # subsample_df = pd.concat([subsample_df[subsample_df["Label"] != "dos_ge_dr"], subsample_df.groupby('Label').apply(pd.DataFrame.sample, frac=0.1).reset_index(drop=True)])
# X = subsample_df.drop(['Label'], axis=1)
# y = subsample_df['Label']

# # Apply PCA
# pca = PCA(n_components=2, random_state=42)
# pca_results = pca.fit_transform(X)

# # Prepare PCA DataFrame for plotting
# pca_df = pd.DataFrame(pca_results, columns=['dimension 1', 'dimension 2'])
# pca_df['Label'] = y.values
# pca_df['Binary'] = y.apply(lambda x: "malicious" if x != 'BENIGN' else 'BENIGN')


# # Apply t-SNE
# tsne = TSNE(n_components=2, n_jobs=-1, verbose=0, random_state=0, perplexity=20)
# tsne_results = tsne.fit_transform(X)

# # Prepare t-SNE DataFrame for plotting
# tsne_df = pd.DataFrame(tsne_results, columns=['dimension 1', 'dimension 2'])
# tsne_df['Label'] = y.values

# # Create figure and axes for the subplots
# fig, axes = plt.subplots(1, 2, figsize=(20, 10))  # Adjust figsize to fit your needs

# # PCA Plot
# sns.scatterplot(x='dimension 1', y='dimension 2', hue='Label',
#                 palette=sns.color_palette('hls', len(pca_df['Label'].value_counts())),
#                 data=pca_df, ax=axes[0])
# axes[0].set_title('CICIDS2017 15 Classes PCA Projection')
# axes[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))
# axes[0].set_ylim(-10, 10)

# # t-SNE Plot
# sns.scatterplot(x='dimension 1', y='dimension 2', hue='Label',
#                 palette=sns.color_palette('hls', len(tsne_df['Label'].value_counts())),
#                 data=tsne_df, ax=axes[1])
# axes[1].set_title('CICIDS2017 15 Classes T-SNE Projection')
# axes[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

# plt.show()

: 

# XGBoost-ShapValues

Data Processing

In [10]:
le = LabelEncoder()
df = flow_df.copy()
df['Label'] = df['Label'].apply(lambda x: 0 if x == "BENIGN" else 1)
df['Label'].value_counts()


Label
1    184906
0     47755
Name: count, dtype: int64

In [11]:
labels = df['Label'].copy()
df.drop(columns='Label', axis=1, inplace=True)
print("Before", df.shape)
df = df[:]

df = df.replace([np.inf, -np.inf], np.nan)  # Replace inf/-inf with NaN
df = df.fillna(df.mean())

df.dropna(inplace=True)
df = df.select_dtypes(include=[np.number])
# df = df.loc[:, (df != df.iloc[0]).any()] #Remove features that are constant
indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
df = df[indices_to_keep]


for i in df.columns:
    df = df[df[i] != "Infinity"]
    df = df[df[i] != np.nan]
    df = df[df[i] != np.inf]
    df = df[df[i] != -np.inf]
    df = df[df[i] != ",,"]
    df = df[df[i] != ", ,"]
    
print(np.any(np.isnan(df)))
print(np.any(np.isfinite(df)))


# Last column turn to binary
# df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: 1 if x != 0 else 0).astype(int)
#Standardize Dataframe
feature_name = list(df.columns)

scaler = StandardScaler()
feature_name = list(df.columns)
normalized_data = scaler.fit_transform(df)
normalized_df = pd.DataFrame(normalized_data, columns=feature_name)
normalized_df['Label'] = labels.values
normalized_df


print("After", df.shape)


X_train, X_test, y_train, y_test = train_test_split(normalized_df.drop(['Label'],axis=1), normalized_df['Label'], test_size=.20, random_state=42)
np.unique(y_train)

Before (232661, 79)
False
True


,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Unnamed: 0.1,Unnamed: 0,Label
0,0.0,-0.653354,-0.24415,-0.208093,-0.016581,-0.225957,0.287503,-0.02887,0.192896,0.581033,...,-0.113087,-0.201698,-0.174232,-0.600189,-0.292625,-0.619362,-0.559182,0.0,0.0,1
1,0.0,-0.651077,-0.24415,-0.208093,-0.017022,-0.225957,0.273313,-0.02887,0.184817,0.558338,...,-0.113087,-0.201698,-0.174232,-0.600189,-0.292625,-0.619362,-0.559182,0.0,0.0,1
2,0.0,-0.651076,-0.24415,-0.208093,-0.016911,-0.225957,0.276860,-0.02887,0.186836,0.564012,...,-0.113087,-0.201698,-0.174232,-0.600189,-0.292625,-0.619362,-0.559182,0.0,0.0,1
3,0.0,-0.651096,-0.24415,-0.208093,-0.014435,-0.225957,0.356676,-0.02887,0.232285,0.691668,...,-0.113087,-0.201698,-0.174232,-0.600189,-0.292625,-0.619362,-0.559182,0.0,0.0,1
4,0.0,-0.651052,-0.24415,-0.208093,-0.017242,-0.225957,0.266218,-0.02887,0.180777,0.546991,...,-0.113087,-0.201698,-0.174232,-0.600189,-0.292625,-0.619362,-0.559182,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232656,0.0,1.210024,-0.22266,-0.099938,-0.035951,-0.224641,-0.336836,-0.02887,-0.162615,-0.417524,...,-0.113087,0.984897,1.289192,0.646187,4.405195,1.253535,-0.011967,0.0,0.0,1
232657,0.0,0.828802,-0.24415,-0.121569,-0.035951,-0.224641,-0.336836,-0.02887,-0.162615,-0.417524,...,-0.113087,0.377653,0.540280,0.392250,3.100454,0.821703,-0.070045,0.0,0.0,1
232658,0.0,0.815913,0.07820,0.289419,0.139259,-0.225957,0.369092,-0.02887,0.215710,0.554727,...,8.558232,4.557952,0.042248,0.310100,2.073020,0.594503,0.004182,0.0,0.0,1
232659,0.0,1.262889,-0.17968,-0.078307,-0.014050,-0.225957,0.369092,-0.02887,-0.001827,0.296550,...,22.137752,14.844042,3.874391,0.422724,3.594914,0.922367,-0.114993,0.0,0.0,1


After (232661, 79)


array([0, 1])

In [12]:
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
# Initialize the CART classifier
cart_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=15, random_state=4022)
cart_classifier.fit(X_train,y_train)
model_outputs = cart_classifier.predict(X_test)
print(f1_score(model_outputs, y_test))

# Train the classifier on the training data

model = XGBClassifier(objective='binary:logistic')
model.fit(X_train,y_train)
model_outputs = model.predict(X_test)
pred = model.score(X_test, y_test)
print(f1_score(model_outputs, y_test))

DecisionTreeClassifier(criterion='entropy', max_depth=15, random_state=4022)

0.9998642705901515


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

0.9999049855442292


In [13]:
import joblib
joblib.dump(model, 'cic_xgb.joblib')
joblib.dump(scaler, 'cic_scaler.joblib')



['cic_xgb.joblib']

['cic_scaler.joblib']

In [14]:
# new_benign_df = pd.read_csv(os.path.join(dataset_csv_path, "BENIGN/final_merged.csv"))
# new_benign_df.drop(columns=["Unnamed: 0.1"], inplace=True)
# features = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
#       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
#       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
#       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std',
#       'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std',
#       'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean',
#       'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot',
#       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
#       'Fwd PSH Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
#       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
#       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'PSH Flag Cnt',
#       'ACK Flag Cnt', 'URG Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
#       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Byts',
#       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
#       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean',
#       'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std',
#       'Idle Max', 'Idle Min']

# X = new_benign_df[features]
# X.replace([np.inf, -np.inf], np.nan, inplace = True)
# X.dropna(inplace=True)
# X_scaled = scaler.transform(X)

# import collections

# output = model.predict(X_scaled) 
# counter = collections.Counter(output)
# counter


SHAP

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(X_train)
np.shape(shap_values.values)

: 

In [ ]:
shap.plots.bar(shap_values, max_display=15)
plt.show()

shap.plots.beeswarm(shap_values, max_display=14)
plt.show()

: 

# Global Model

### Functions

In [15]:
def get_optimal_threshold(precision, recall, thresholds):
    with np.errstate(divide='ignore', invalid='ignore'):
        f1_scores = 2 * (precision * recall) / (precision + recall)
        f1_scores[np.isnan(f1_scores)] = 0  # Replace NaN values with 0    
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold
  
  
def eval_accuracy_from_scores(anomaly_scores, y_test):
    fpr, tpr, _ = roc_curve(y_test, anomaly_scores)
    precision, recall, thresholds = precision_recall_curve(y_test, anomaly_scores)
    opt_threshold = get_optimal_threshold(precision, recall, thresholds)
    pred = np.where(anomaly_scores > opt_threshold, 1,0)
    f1 = f1_score(y_test, pred)
    conf_matrix = confusion_matrix(y_test, pred)
    print(f"F1 Score: {f1:.4f}")
    print(conf_matrix)
    
    
    
def set_supervised_label(supervision_rate, anomaly_rate, y):
        idx = np.where(y_train == 1)[0]
        y = np.zeros_like(y.values)
        if supervision_rate == 0:
            print(f"UNSUPERVISED, Sampling Rate = {anomaly_rate}")
            y[idx[:2]] = 1
        else:
            print(f"SEMI SUPERVISED, Sampling Rate = {anomaly_rate}, Supervision = {supervision_rate}")
            idx = np.random.choice(idx, size=int(supervision_rate * len(idx)), replace=False)
            y[idx] = 1
        return y

In [16]:
model = PReNet
clf = model( device = 'cuda', verbose=1, epochs=1)
clf.fit(X=X_train.to_numpy()[:10000], y = y_train[:10000])
anomaly_scores = clf.decision_function(X_test.to_numpy())
eval_accuracy_from_scores(anomaly_scores, y_test)


Start Training...
ensemble size: 1
epoch  1, training loss: 1.230963, time: 1.9s
Start Inference on the training data...


F1 Score: 0.9880
[[ 9099   597]
 [  290 36547]]


In [ ]:
# from collections import Counter
# model = PReNet
# clf = model( device = 'cuda', verbose=1, epochs=1)
# y_train_sampled = set_supervised_label(0.02, 0.02, y_train)
# counter = Counter(y_train_sampled)
# print(counter)
# clf.fit(X=X_train.to_numpy(), y = y_train_sampled)
# anomaly_scores = clf.decision_function(X_test.to_numpy())
# eval_accuracy_from_scores(anomaly_scores, y_test)

: 

In [17]:
clf.to('cpu')

AttributeError: 'PReNet' object has no attribute 'to'

In [18]:
# joblib.dump(clf,"PReNet_100.joblib")
import pickle
pickle.dump(clf, open("PReNet_100.pkl", "wb"))

OSError: [Errno 28] No space left on device

In [ ]:
clf

: 